 # AtCoder Beginner Contest 177

 [コンテストトップページ](https://atcoder.jp/contests/abc177)

 # A
 ## 提出コード


In [ ]:
inputs = input().split()
D = int(inputs[0])
T = int(inputs[1])
S = int(inputs[2])

print("Yes" if D / S <= T else "No")


 ## 公式解説

 Python以外の言語だと `D/S` のように整数同士で除算すると、結果が小数点以下切り捨ての整数になってしまうので注意。
 これを避けるには，Dと `S*T`（答えが問題なく整数になるかけ算）の大きさを比較するようにすると良い．

 # B
 ## 提出コード

In [ ]:
S = input()
T = input()

min_count = len(T)
for s_i in range(len(S)):
    i = s_i
    matched = 0
    for j in range(len(T)):
        if len(S) <= i:  # Out of S
            matched = 0  # Cannot append characters to S
            break
        if S[i] == T[j]:
            matched += 1
        i += 1
    count = len(T) - matched
    if count < min_count:
        min_count = count
print(min_count)


 ## 公式解説

 同じ全探索（計算量はO(|S||T|)）だけど、
 回答例の方が添字の扱いがもっとお上手だった。

 ## 公式解説を受けてコード


In [ ]:
S = input()
T = input()

min_count = len(T)
for s in range(len(S) - len(T) + 1):  # T cannot exceed the length of S
    matched = 0
    for i in range(len(T)):
        # print(f'S[{s + i}] == T[{i}]')
        if S[s + i] == T[i]:
            matched += 1
    count = len(T) - matched
    if count < min_count:
        min_count = count
print(min_count)


 # C

 ## 提出コード（TLE）

In [ ]:
N = int(input())
A = [int(x) for x in input().split()]

sum = 0
for i in range(N):
    for j in range(i + 1, N):
        sum += A[i] * A[j]
print(sum % (10 ** 9 + 7))


 ## 公式解説

 N < 2 x 10^5 なので、O(N^2)は間に合わない。

 式を変形すると、累積和をあらかじめ求めておくか、
 または前のループの結果を使って区間和を求めておけば良い問題になるので、
 O(N)で解ける。

 ## 解説を踏まえて書き直したコード


In [ ]:
N = int(input())
A = [int(x) for x in input().split()]

sums = [0] * N
sums[0] = A[0]
for i in range(N - 1):
    sums[i + 1] = sums[i] + A[i + 1]  # sums[i] stores the sum of A0~Ai

sum = 0
for i in range(N):
    sum += A[i] * (sums[N - 1] - sums[i])

print(sum % 1000000007)


 # D

 ## 解説を踏まえて書いたコード（Union-Find木利用）

In [ ]:
class UnionFind:
    def __init__(self, N: int):
        # the parent of the root stores negative number of its size (-size)
        self.parents = [-1] * N
        self.ranks = [-1] * N
        for i in range(N):
            self.ranks[i] = 0

    def find_root(self, x):
        if self.parents[x] < 0:
            return x
        else:
            # find the root of x and make a shortcut pass
            self.parents[x] = self.find_root(self.parents[x])
            return self.parents[x]

    def unite(self, x, y):
        x = self.find_root(x)
        size_x = -self.parents[x]
        y = self.find_root(y)
        size_y = -self.parents[y]
        if self.ranks[x] < self.ranks[y]:
            self.parents[x] = y  # merge the tree under x to y
            self.parents[y] = -(size_x + size_y)
        else:
            self.parents[y] = x  # merge the tree under y to x
            self.parents[x] = -(size_x + size_y)
            # Rank of x increments (Note that the rank of the root doesn't change otherwise)
            if self.ranks[x] == self.ranks[y]:
                self.ranks[x] += 1

    def same(self, x, y):
        return self.find_root(x) == self.find_root(y)

    def size(self, x):
        x = self.find_root(x)
        return -self.parents[x]


def get_input():
    N, M = (int(x) for x in input().split())
    rels = []
    for m in range(M):
        r = [int(x) - 1 for x in input().split()]  # adjust to 0-origin
        rels.append(r)
    return N, M, rels


N, M, rels = get_input()

# Map friends on a union-find tree
tree = UnionFind(N)
for r in rels:
    x = r[0]
    y = r[1]
    if not tree.same(x, y):
        tree.unite(x, y)
# print(tree.parents)

# Max number of groups is the size of the largest group
max_size = 0
for i in range(N):
    s = tree.size(i)
    if max_size < s:
        max_size = s
print(max_size)


 # E


In [ ]:
# ## 公式解説
#
# N <= 10^6 なので、O(N^2)のアルゴリズムは間に合わない。
#
# エラトステネスのふるいを使って、全てのAiについて各最小の素因子を求めておく。
# （この計算はA=Aiの最大値としたとき、O(AloglogA + NlogA)）
# そしてAiを素因数分解して、共通の素因子が無ければpairwise、
# 2つ以上のAiに共通の素因子が無ければsetwise、全てのAiに共通の素因氏があればnot coprime。


 ## 解説を踏まえた実装

 Python 3.8だと数問TLEになるので、PyPy3選択。


In [ ]:
import math


def get_pseudo_input():
    N = 3
    A = [6, 10, 16]
    return N, A


def get_input():
    N = int(input())
    A = [int(x) for x in input().split()]
    return N, A


def sieve_of_eratosthenes(x: int):
    ret = [-1] * (x + 1)
    for i in range(1, x + 1):
        ret[i] = i
    p_max = int(math.sqrt(x))
    for p in range(2, p_max + 1):
        if ret[p] < p:  # p is not a prime number
            continue
        for i in range(p, x + 1, p):
            if ret[i] > p:
                ret[i] = p
    return ret


A_max = 10 ** 6
p_table = sieve_of_eratosthenes(A_max)

N, A = get_input()
# Store a number of unique Ais using the prime number
p_nums = [0] * (A_max + 1)
p_nums[1] = N
max_p_nums = 0
for a in A:
    # print(f"a={a}")
    while a > 1:
        p = p_table[a]
        while a % p == 0:
            a = a // p
        # print(f"  a={a}")
        p_nums[p] += 1
        if max_p_nums < p_nums[p]:
            max_p_nums = p_nums[p]
# print(max_p_nums)
if max_p_nums <= 1:
    # GCD of A is 1
    print('pairwise coprime')
elif max_p_nums == N:
    # GCD of A is not 1
    print('not coprime')
else:
    print('setwise coprime')
